In [4]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# 1. Cargar la API Key desde el archivo .env
load_dotenv() 
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

def ejecutar_agente_tendencia():
    # 2. Carga de datos
    df = pd.read_csv("data/raw_data/ethereum_1d_indicators_raw.csv", index_col=0, parse_dates=True)
    
    # Seleccionamos los últimos 7 días para dar contexto de tendencia
    ultimos_7_dias = df.tail(7).copy()
    
    # Convertimos los datos a un formato de texto amigable (Markdown) para el prompt
    # Esto ayuda al LLM a leerlo como una tabla
    tabla_datos = ultimos_7_dias[['Close', 'RSI', 'MACD_Hist', 'BB_High']].to_markdown()

    # 3. Prompt Mejorado
    prompt = f"""
    Eres un analista financiero experto en Ethereum. Se te proporcionan los datos técnicos de los últimos 7 días.
    
    HISTÓRICO RECIENTE (Últimos 7 días):
    {tabla_datos}
    
    TAREA:
    1. Presenta un cuadro resumen con los valores más relevantes del día de hoy.
    2. Analiza la TENDENCIA: ¿El RSI ha estado subiendo o bajando en esta semana? ¿El precio se está acercando a la Banda Superior de Bollinger?
    3. Dame tu opinión profesional sobre si es conveniente entrar en SHORT ahora mismo.
    
    REGLAS TÉCNICAS:
    - Si el RSI pasó de 50 a 70 en 7 días, hay un impulso alcista fuerte (cuidado con el short prematuro).
    - Si el RSI está en 30 o menos y bajando, es sobreventa extrema.
    - El cuadro de valores debe ser claro y fácil de leer.
    """

    # 4. Llamada al modelo
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un Agente de Data & Finance. Respondes con cuadros técnicos y análisis crítico de tendencias."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content

# 5. Ejecución
print(ejecutar_agente_tendencia())

### Cuadro Resumen del Día de Hoy (2026-02-15)

| Métrica    | Valor   |
|------------|---------|
| Cierre     | 2060.83 |
| RSI        | 35.7893 |
| MACD_Hist  | 14.3299 |
| BB_High    | 2953.42 |

### Análisis de Tendencia

1. **RSI (Relative Strength Index):**
   - El RSI ha mostrado un ligero incremento desde un mínimo de 28.5068 el 2026-02-11 a 35.7893 el 2026-02-15.
   - Aunque el RSI ha subido, se mantiene por debajo de 50, lo que indica que el mercado aún no ha entrado en territorio de sobrecompra. Sin embargo, está saliendo de una zona de sobreventa.

2. **Precio vs. Banda Superior de Bollinger:**
   - El precio de cierre actual (2060.83) está significativamente por debajo de la Banda Superior de Bollinger (2953.42).
   - Esto sugiere que el precio no se está acercando a la Banda Superior, lo que indica que no hay una presión alcista fuerte en este momento.

### Opinión Profesional sobre Entrar en SHORT

- **RSI y MACD:**
  - El RSI ha estado subiendo desde niveles de sobreven

In [6]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# 1. Cargar configuración
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
TOKEN = os.getenv("TELEGRAM_TOKEN")
CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")

def analizar_tendencia_7_dias():
    """Carga los datos completos y genera un análisis técnico avanzado"""
    try:
        # Cargamos el CSV con todos los indicadores
        df = pd.read_csv("data/raw_data/ethereum_1d_indicators_raw.csv", index_col=0, parse_dates=True)
        
        # Seleccionamos los últimos 7 días
        ultimos_7 = df.tail(7)
        
        # Incluimos TODAS las columnas generadas en data_indicators.ipynb
        # 'Close', 'RSI', 'BB_High', 'BB_Low', 'BB_Mid', 'MACD_Hist', 'ATR', etc.
        datos_formateados = ultimos_7.to_markdown()
        
        prompt = f"""
        Eres un Agente de Trading Pro de Ethereum. Analiza el siguiente set de datos técnicos:
        
        {datos_formateados}
        
        TAREA:
        1. 💎 **ESTADO ACTUAL**: Bloque de código con Precio, RSI, MACD_Hist y ATR.
        2. 📈 **ANÁLISIS TÉCNICO**: 
           - Comenta la posición del precio respecto a las Bandas de Bollinger (BB_High, BB_Low).
           - Analiza la fuerza de la tendencia usando el MACD y el RSI.
        3. 🚨 **VEREDICTO**: SHORT, WAIT o LONG con justificación técnica.
        4. 🛡️ **GESTIÓN DE RIESGO**: 
           - Sugiere un Stop Loss basado en el ATR (ej: Precio + 2*ATR para Shorts).
           - Sugiere un Take Profit usando el BB_Mid o BB_Low.
        
        Usa Emojis y bloques de código para que el reporte sea visualmente impecable en Telegram.
        """
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "Eres un analista financiero senior. Tu estilo es visual, técnico y directo."},
                      {"role": "user", "content": prompt}],
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Error al procesar datos: {e}"

def enviar_telegram(texto):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    requests.post(url, data={"chat_id": CHAT_ID, "text": texto, "parse_mode": "Markdown"})

# --- BUCLE PRINCIPAL ---
print("🤖 Bot activo en @aiagent_ivo196_bot...")
print("Escríbele /analisis o /short para recibir el reporte.")

last_update_id = 0
while True:
    try:
        url = f"https://api.telegram.org/bot{TOKEN}/getUpdates?offset={last_update_id + 1}"
        updates = requests.get(url).json()

        for update in updates.get("result", []):
            last_update_id = update["update_id"]
            texto_usuario = update["message"].get("text", "").lower()

            if texto_usuario in ["/analisis", "/short", "hola"]:
                enviar_telegram("📊 Analizando indicadores de Ethereum de los últimos 7 días...")
                # Llamamos a la función que usa el LLM
                resultado = analizar_tendencia_7_dias()
                enviar_telegram(resultado)
        
        time.sleep(2)
    except KeyboardInterrupt:
        print("Bot apagado.")
        break
    except Exception as e:
        print(f"Error en el bucle: {e}")
        time.sleep(5)

🤖 Bot activo en @aiagent_ivo196_bot...
Escríbele /analisis o /short para recibir el reporte.
Bot apagado.
